# Wind Farm Spotter: Inference Test (Batch, Single) [Python, GPU Runtime]

### **Environment Setup**
First setting up the colab/jupyter notebook with necessary packages; also mounting Google Drive for our external working directory/file store.

In [0]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Install pytorchcv** package to have access to base architecture implementations.

In [0]:
!pip install pytorchcv

**Imports:**

In [0]:
import fastai
from fastai.vision import *
from pytorchcv.model_provider import get_model as ptcv_get_model
import warnings
warnings.filterwarnings('ignore')

In [0]:
print(fastai.__version__)

1.0.59


### **Test Set Data Loaders**
Next we'll create Data Loaders for both EfficientNet-b1 and Inception-v3 to perform inference. 

In [0]:
path = Path("/content/drive/My Drive/dla-project/data/")
path.ls()

[PosixPath('/content/drive/My Drive/dla-project/data/test'),
 PosixPath('/content/drive/My Drive/dla-project/data/train'),
 PosixPath('/content/drive/My Drive/dla-project/data/test_single'),
 PosixPath('/content/drive/My Drive/dla-project/data/effnet-b1-c9-sd'),
 PosixPath('/content/drive/My Drive/dla-project/data/effnet-b1-sd.pth'),
 PosixPath('/content/drive/My Drive/dla-project/data/incept-v3-80-sd.pth'),
 PosixPath('/content/drive/My Drive/dla-project/data/wfs-incept-sd1.pth'),
 PosixPath('/content/drive/My Drive/dla-project/data/wfs-incept-sd2.pth'),
 PosixPath('/content/drive/My Drive/dla-project/data/wfs-incept-sd3.pth'),
 PosixPath('/content/drive/My Drive/dla-project/data/wfs-incept-sd4.pth'),
 PosixPath('/content/drive/My Drive/dla-project/data/wfs-incept-sd5.pth'),
 PosixPath('/content/drive/My Drive/dla-project/data/wfs-incept-sd6.pth'),
 PosixPath('/content/drive/My Drive/dla-project/data/wfs-incept-sd7.pth'),
 PosixPath('/content/drive/My Drive/dla-project/data/wfs-incept

Since we want to be able to tabulate the accuracy of our predictions, we need to use a labeled test set. The directory entitled test is a labeled set, while the directory entitled test_single is unlabeled.

In [0]:
bs = 32 
size1 = 320
size2 = 363
tfms = ([crop_pad()], [])

In [0]:
il = ImageList.from_folder(path/'test')
ils = il.split_none()
ll = ils.label_from_folder()
ll.valid = ll.train
ll.test = ll.train

**Inference Data Loader for EfficientNet:**

In [0]:
effnet_ll = ll 
effnet_ll.transform(tfms=tfms,size=size1)
effnet_data = effnet_ll.databunch(bs=bs);
effnet_data.normalize(imagenet_stats)

ImageDataBunch;

Train: LabelList (49 items)
x: ImageList
Image (3, 320, 320),Image (3, 320, 320),Image (3, 320, 320),Image (3, 320, 320),Image (3, 320, 320)
y: CategoryList
no_turbines_high_potential,no_turbines_high_potential,no_turbines_high_potential,no_turbines_high_potential,no_turbines_high_potential
Path: /content/drive/My Drive/dla-project/data/test;

Valid: LabelList (49 items)
x: ImageList
Image (3, 320, 320),Image (3, 320, 320),Image (3, 320, 320),Image (3, 320, 320),Image (3, 320, 320)
y: CategoryList
no_turbines_high_potential,no_turbines_high_potential,no_turbines_high_potential,no_turbines_high_potential,no_turbines_high_potential
Path: /content/drive/My Drive/dla-project/data/test;

Test: LabelList (49 items)
x: ImageList
Image (3, 320, 320),Image (3, 320, 320),Image (3, 320, 320),Image (3, 320, 320),Image (3, 320, 320)
y: CategoryList
no_turbines_high_potential,no_turbines_high_potential,no_turbines_high_potential,no_turbines_high_potential,no_turbines_high_potential


**Inference Data Loader for Inception:**

In [0]:
incept_ll = ll 
incept_ll.transform(tfms=tfms,size=size2)
incept_data = incept_ll.databunch(bs=bs);
incept_data.normalize(imagenet_stats)

ImageDataBunch;

Train: LabelList (49 items)
x: ImageList
Image (3, 363, 363),Image (3, 363, 363),Image (3, 363, 363),Image (3, 363, 363),Image (3, 363, 363)
y: CategoryList
no_turbines_high_potential,no_turbines_high_potential,no_turbines_high_potential,no_turbines_high_potential,no_turbines_high_potential
Path: /content/drive/My Drive/dla-project/data/test;

Valid: LabelList (49 items)
x: ImageList
Image (3, 363, 363),Image (3, 363, 363),Image (3, 363, 363),Image (3, 363, 363),Image (3, 363, 363)
y: CategoryList
no_turbines_high_potential,no_turbines_high_potential,no_turbines_high_potential,no_turbines_high_potential,no_turbines_high_potential
Path: /content/drive/My Drive/dla-project/data/test;

Test: LabelList (49 items)
x: ImageList
Image (3, 363, 363),Image (3, 363, 363),Image (3, 363, 363),Image (3, 363, 363),Image (3, 363, 363)
y: CategoryList
no_turbines_high_potential,no_turbines_high_potential,no_turbines_high_potential,no_turbines_high_potential,no_turbines_high_potential


### **Reloading the Fast.ai Learner**
Fast.ai provides a useful method named **load_learner()** to reinstantiate an instance model from a pickle file. The loaded learner recalls all the transformations, and parameters used during training:

In [0]:
inference_test_efficientnet = load_learner(path, file=path/'train/effnet-b1-c98.pkl')
inference_test_inceptionv3 = load_learner(path, file=path/'train/incept-v3-95-2.pkl')

We'll then attach our data to the loaded learners:

In [0]:
inference_test_efficientnet.data = effnet_data
inference_test_inceptionv3.data = incept_data

### **Inference Testing (Batch)**
The Fast.ai learner comes with a handy method called **get_preds()** that enables you to run batch inference on either the training, validation, or test set. 

We'll be timing our test to compare with other runtimes and using top 1/3/5% as our performance metrics. 

In [0]:
print(fastai.vision.defaults.device)

cuda


**EfficientNet Batch Inference:**

In [0]:
gc.collect()

0

In [0]:
%time effnet_results = inference_test_efficientnet.get_preds(ds_type=DatasetType.Test)

CPU times: user 178 ms, sys: 172 ms, total: 350 ms
Wall time: 12.2 s


**Performance**:

In [0]:
effnet_top_1 = top_k_accuracy(effnet_results[0], effnet_results[1], k=1)
effnet_top_3 = top_k_accuracy(effnet_results[0], effnet_results[1], k=3)
effnet_top_5 = top_k_accuracy(effnet_results[0], effnet_results[1], k=5)

print("Top 1% Accuracy: ", effnet_top_1)
print("Top 3% Accuracy: ", effnet_top_3)
print("Top 5% Accuracy: ", effnet_top_5)

Top 1% Accuracy:  tensor(0.7347)
Top 3% Accuracy:  tensor(0.9184)
Top 5% Accuracy:  tensor(0.9796)


**Results:**

In [0]:
inference_test_efficientnet.show_results(rows=9)

Output hidden; open in https://colab.research.google.com to view.

**Inceptionv3 Batch Inference:**

In [0]:
gc.collect()

53975

In [0]:
%time incept_results = inference_test_inceptionv3.get_preds(ds_type=DatasetType.Test)

CPU times: user 172 ms, sys: 167 ms, total: 340 ms
Wall time: 12.4 s


**Performance**:

In [0]:
incept_top_1 = top_k_accuracy(incept_results[0], incept_results[1], k=1)
incept_top_3 = top_k_accuracy(incept_results[0], incept_results[1], k=3)
incept_top_5 = top_k_accuracy(incept_results[0], incept_results[1], k=5)

print("Top 1% Accuracy: ", incept_top_1)
print("Top 3% Accuracy: ", incept_top_3)
print("Top 5% Accuracy: ", incept_top_5)

Top 1% Accuracy:  tensor(0.6939)
Top 3% Accuracy:  tensor(0.9592)
Top 5% Accuracy:  tensor(0.9796)


**Results:**

In [0]:
inference_test_inceptionv3.show_results(rows=9)

Output hidden; open in https://colab.research.google.com to view.

### **Inference Testing (Single)**
Fast.ai's **load_learner()** method also gives us the option of passing a list of images (unlabeled) right from the start. To run our Single Image Inference Test, we'll use the single_test directory to create a Fast.ai **ImageList**.

In [0]:
inference_effnet_single = load_learner(path, file=path/'train/effnet-b1-c98.pkl', test=ImageList.from_folder(path/"test_single"))
inference_incept_single = load_learner(path, file=path/'train/incept-v3-95-2.pkl', test=ImageList.from_folder(path/"test_single"))


In [0]:
test_files = path/"test_single"
test_name_list = os.listdir(test_files)
#test_name_list[:5]

**EfficientNet Single Inference:**

In [0]:
gc.collect()

53985

In [0]:
inference_effnet_single.model.eval()

In [0]:
import time
preds = {}
start = time.time()
for i in range(0, len(test_name_list)):
  img_name = str(test_name_list[i])
  img = inference_effnet_single.data.test_dl.dl.dataset.x[i]
  p = inference_effnet_single.predict(img)
  preds[img_name] = str(p[0]); 
end = time.time()
pred_list = list(preds.items())
print("elapsed: ", end - start)

elapsed:  15.133553266525269


**Results:**

In [0]:
results = pd.DataFrame(pred_list, columns=["file_name", "id"])
print(results)

                                file_name                          id
0                test_high_capacity_1.jpg      turbines_high_capacity
1                test_high_capacity_2.jpg      turbines_high_capacity
2    test_no_turbines_med_potential_2.jpg   no_turbines_med_potential
3                test_high_capacity_3.jpg      turbines_high_capacity
4     test_no_turbines_no_potential_1.jpg    no_turbines_no_potential
5                test_high_capacity_4.jpg      turbines_high_capacity
6                test_high_capacity_5.jpg      turbines_high_capacity
7                test_high_capacity_6.jpg      turbines_high_capacity
8                test_high_capacity_7.jpg      turbines_high_capacity
9                test_high_capacity_8.jpg      turbines_high_capacity
10                test_low_capacity_1.jpg   no_turbines_low_potential
11                test_low_capacity_2.jpg       turbines_low_capacity
12                test_low_capacity_3.jpg       turbines_low_capacity
13                te

In [0]:
gc.collect()

186

**Inception-v3 Single Inference:**

In [0]:
inference_incept_single.model.eval()

In [0]:
preds = {}
start = time.time()
for i in range(0, len(test_name_list)):
  img_name = str(test_name_list[i])
  img = inference_incept_single.data.test_dl.dl.dataset.x[i]
  p = inference_incept_single.predict(img)
  preds[img_name] = str(p[0]); 
end = time.time()
pred_list = list(preds.items())
print("elapsed: ", end - start)

elapsed:  15.491671800613403


**Results:**

In [0]:
results = pd.DataFrame(pred_list, columns=["file_name", "id"])
print(results)

                                file_name                          id
0                test_high_capacity_1.jpg      turbines_high_capacity
1                test_high_capacity_2.jpg    turbines_medium_capacity
2    test_no_turbines_med_potential_2.jpg   no_turbines_low_potential
3                test_high_capacity_3.jpg    turbines_medium_capacity
4     test_no_turbines_no_potential_1.jpg    no_turbines_no_potential
5                test_high_capacity_4.jpg      turbines_high_capacity
6                test_high_capacity_5.jpg      turbines_high_capacity
7                test_high_capacity_6.jpg    turbines_medium_capacity
8                test_high_capacity_7.jpg      turbines_high_capacity
9                test_high_capacity_8.jpg      turbines_high_capacity
10                test_low_capacity_1.jpg       turbines_low_capacity
11                test_low_capacity_2.jpg       turbines_low_capacity
12                test_low_capacity_3.jpg       turbines_low_capacity
13                te